In [ ]:
import brightway2 as bw
import bw2regional as bwr
from bw2regional.pandarus_remote import remote, AlreadyExists
import bw2_aware
import rower
import os
from bw2analyzer import ContributionAnalysis

In [ ]:
bw.projects.set_current("DF69 Case Study")

# Import ecoinvent 3.5

In [ ]:
bw.create_default_biosphere3()

In [ ]:
bw.create_core_migrations()

In [ ]:
ei = bw.SingleOutputEcospold2Importer(
    "/Users/cmutel/Documents/LCA/Ecoinvent/3.5/cutoff/datasets",
    "ecoinvent 3.5 cutoff"
)
ei.apply_strategies()
ei.statistics()

In [ ]:
ei.write_database()

## Fix bug in brightway

We are incorrectly accounting for water balances. Need to set water production values to negative, water consumption to positive (because CFs, representing damage, are positive).

In [ ]:
water_production = [
    x for x in bw.Database("biosphere3")
    if 'water' in x['name'].lower() and x['categories'][0] == 'water'
]

In [ ]:
water_production_codes = [x['code'] for x in water_production]

In [ ]:
from bw2data.backends.peewee.proxies import *

In [ ]:
exc_qs = ExchangeDataset.select().where(
    ExchangeDataset.input_code << tuple(water_production_codes)
)

len(exc_qs)

In [ ]:
for exc in exc_qs:
    exc.data['amount'] *= -1
    exc.save()

In [ ]:
bw.Database("ecoinvent 3.5 cutoff").process()

# Import case study inventory

In [ ]:
imp = bw.ExcelImporter("rice_inventory.xlsx")
imp.apply_strategies()
imp.match_database("ecoinvent 3.5 cutoff", fields=('name', 'unit', 'location', 'reference product'))
imp.match_database("ecoinvent 3.5 cutoff", fields=('name', 'unit', 'location'))
imp.match_database(fields=['name', 'location'])
imp.statistics()

In [ ]:
imp.write_database()

# Fix geographic data in LCI databases

## Install basic world definitions

In [ ]:
bwr.bw2regionalsetup()

## Define rest of world locations in ecoinvent

In [ ]:
rower_obj = rower.Rower('ecoinvent 3.5 cutoff')
rower_obj.load_existing(rower_obj.EI_3_5_CUTOFF)
rower_obj.label_RoWs()

## Label locations with their "geocollection"

Need to turn e.g. `"Europe without CH"` into `("ecoinvent", "Europe without CH")`.

In [ ]:
bwr.label_activity_geocollections("ecoinvent 3.5 cutoff")
bwr.label_activity_geocollections("AWARE Case Study")

In [ ]:
bw.databases['ecoinvent 3.5 cutoff']['geocollections'] = ['world', 'ecoinvent', 'RoW']
bw.databases['AWARE Case Study']['geocollections'] = ['world', 'ecoinvent', 'RoW']
bw.databases['biosphere3']['geocollections'] = []
bw.databases.flush()

# Import AWARE

In [ ]:
bw2_aware.import_aware()

# Add extension tables

## Define where rice is grown

Our rice inventory is country-specific, but this is too general for China and the US. We use data from Ramankutty on global rice production. Here is the map for part of Asia:

<img src="images/rice-asia.png">

In [ ]:
fp = os.path.abspath("rice.tiff")
assert os.path.isfile(fp)

bwr.geocollections['rice'] = {
    'filepath': fp,
    'band': 1
}

## Add population density

We assume that the spatial pattern of other water-consuming activities follows population density.

In [ ]:
fp = os.path.abspath("gdpweighted.tif")
assert os.path.isfile(fp)

bwr.geocollections['weighted-pop-density'] = {
    'filepath': fp,
    'band': 1
}

## Calculate raster statistics for our extension tables

In [ ]:
crop_rasters = [
    'rice',
    'weighted-pop-density',
]

for raster in crop_rasters:
    try:
        remote.calculate_rasterstats('world-topo-watersheds-aware', raster)
    except AlreadyExists:
        pass

In [ ]:
for raster in crop_rasters:
    remote.rasterstats_as_xt('world-topo-watersheds-aware', raster, "xt-" + raster)

# LCA calculations

We will split our calculations into the direct impact from irrigation, which will use the agricultural CFs, and everything else, which won't. We start by looking at the site-generic score and major contributors.

In [ ]:
swiss_rice = ('AWARE Case Study', 'Cooked rice CH')
chinese_rice = ('AWARE Case Study', 'Cooked rice CN')

In [ ]:
irrigation = [x for x in bw.Database("ecoinvent 3.5 cutoff") 
              if x['name'] == 'irrigation']

rice_production = [x for x in bw.Database("ecoinvent 3.5 cutoff") 
              if x['name'] == 'rice production']

## Rice in China

In [ ]:
background_lca_cn = bwr.ExtensionTablesLCA(
    {chinese_rice: 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Non-agricultural'),
    xtable='xt-weighted-pop-density',
    limitations={
        'activities': irrigation + rice_production,
        'activities mode': 'exclude'
    }
)
background_lca_cn.lci()
background_lca_cn.lcia()
background_lca_cn.score

In [ ]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(background_lca_cn)[:10]:
        print([act['name'], act['unit'], act['reference product'], 
               act['location'], score, score / background_lca_cn.score, amount])

In [ ]:
foreground_lca_cn = bwr.ExtensionTablesLCA(
    {chinese_rice: 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Agricultural'),
    xtable='xt-rice',
    limitations={
        'activities': irrigation + rice_production,
        'activities mode': 'include'
    }
)
foreground_lca_cn.lci()
foreground_lca_cn.lcia()
foreground_lca_cn.score

In [ ]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(foreground_lca_cn)[:10]:
        print([act['name'], act['unit'], act['reference product'], 
               act['location'], score, score / foreground_lca_cn.score, amount])

## Rice in Switzerland

In [ ]:
background_lca_ch = bwr.ExtensionTablesLCA(
    {swiss_rice: 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Non-agricultural'),
    xtable='xt-weighted-pop-density',
    limitations={
        'activities': irrigation + rice_production,
        'activities mode': 'exclude'
    }
)
background_lca_ch.lci()
background_lca_ch.lcia()
background_lca_ch.score

In [ ]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(background_lca_ch)[:10]:
        print([act['name'], act['unit'], act['reference product'], 
               act['location'], score, score / background_lca_ch.score, amount])

In [ ]:
foreground_lca_ch = bwr.ExtensionTablesLCA(
    {swiss_rice: 1},
    ('AWARE', '1.2 (April 2017)', 'Annual', 'Agricultural'),
    xtable='xt-rice',
    limitations={
        'activities': irrigation + rice_production,
        'activities mode': 'include'
    }
)
foreground_lca_ch.lci()
foreground_lca_ch.lcia()
foreground_lca_ch.score

In [ ]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(foreground_lca_ch)[:10]:
        print([act['name'], act['unit'], act['reference product'], 
               act['location'], score, score / foreground_lca_ch.score, amount])